<a href="https://colab.research.google.com/github/GrigoreGeorgeAlexandru/Colab-projects/blob/main/Laborator7_A_Data_Transformations_stud.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Transformări de date

Datele nu se află întotdeauna într-un format convenabil, prin urmare acestea trebuie procesate. De multe ori, trebuie procesate date non-numerice (nume de clienți, coduri poștale, nume de țări etc.)

O parte importantă a lucrului cu date este cea de înțelegere a acestora. Folosind propriile cunoștințe asupra domeniului, se construiește o intuiție a modului în care datele respective trebuie abordate și procesate. Uneori, cel mai bun lucru este ștergerea anumitor date, alteori trebuie realizată o etapă de feature-engineering. În general, se încearcă transformarea datelor într-un format adecvat datelor de intrare pentru algoritmii ML.

Spark oferă metode ce permit realizarea acestor transformări: http://spark.apache.org/docs/latest/ml-features.html


In [ ]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 1.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=e0916fbc5588c3a3690f58195d0132616684d0e1cc01e56498ed00ddef2809b5
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [ ]:
#Import SparkSession
from pyspark.sql import SparkSession


In [ ]:
#Construire sesiune Spark
spark=SparkSession.builder.master("local[*]").appName("Data Transformation").getOrCreate()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#Citirea fisierului fake_customers.csv in DataFrame
drive_path="/content/drive/MyDrive/Data"
df=spark.read.csv(path=drive_path+"/fake_customers.csv",header=True, inferSchema=True)

In [ ]:
#Afișare
df.show()

+-------+----------+-----+
|   Name|     Phone|Group|
+-------+----------+-----+
|   John|4085552424|    A|
|   Mike|3105552738|    B|
| Cassie|4085552424|    B|
|  Laura|3105552438|    B|
|  Sarah|4085551234|    A|
|  David|3105557463|    C|
|   Zach|4085553987|    C|
|  Kiera|3105552938|    A|
|  Alexa|4085559467|    C|
|Karissa|3105553475|    A|
+-------+----------+-----+



In [ ]:
df.count()

10

## Caracteristici ale datelor

### StringIndexer

De multe ori va trebui să convertim informația de tip șir de caractere în informație numerică, ce corespunde caracteristicilor categoriale. Această transformare se realizează cu ușurință cu ajutorul clasei StringIndexer:

In [ ]:
from pyspark.ml.feature import StringIndexer

df1 = spark.createDataFrame(
    [(0, "a"), (1, "b"), (2, "c"), (3, "a"), (4, "a"), (5, "c")],
    ["user_id", "category"])

indexer = StringIndexer(inputCol="category", outputCol="categoryIndex")
indexed = indexer.fit(df1).transform(df1)
indexed.show()

+-------+--------+-------------+
|user_id|category|categoryIndex|
+-------+--------+-------------+
|      0|       a|          0.0|
|      1|       b|          2.0|
|      2|       c|          1.0|
|      3|       a|          0.0|
|      4|       a|          0.0|
|      5|       c|          1.0|
+-------+--------+-------------+



In [ ]:
indexer_customers = StringIndexer(inputCol="Group", outputCol="groupIndex")
indexed_c=indexer_customers.fit(df).transform(df)
indexed_c.show()


+-------+----------+-----+----------+
|   Name|     Phone|Group|groupIndex|
+-------+----------+-----+----------+
|   John|4085552424|    A|       0.0|
|   Mike|3105552738|    B|       1.0|
| Cassie|4085552424|    B|       1.0|
|  Laura|3105552438|    B|       1.0|
|  Sarah|4085551234|    A|       0.0|
|  David|3105557463|    C|       2.0|
|   Zach|4085553987|    C|       2.0|
|  Kiera|3105552938|    A|       0.0|
|  Alexa|4085559467|    C|       2.0|
|Karissa|3105553475|    A|       0.0|
+-------+----------+-----+----------+



### VectorAssembler

VectorAssembler este un transformator ce combină o listă dată de coloane în cadrul unui singur vector coloană. Acesta este util pentru combinarea caracteristicilor brute și a celor generate de către diferiți transformatori într-un singur vector de caracteristici, cu scopul de a antrena modele ML (regresie logistică, arbori de decizie etc.)

VectorAssembler acceptă următoarele tipuri ale coloanelor de intrare: toate tipurile numerice, tipul boolean și vector. În fiecare linie, valorile coloanelor de intrare vor fi concatenate într-un vector, în ordinea specificată.

Presupunem că avem un DataFrame cu coloanele "id", "hour", "mobile", "userFeatures" și "clicked":

     id | hour | mobile | userFeatures     | clicked
    ----|------|--------|------------------|---------
     0  | 18   | 1.0    | [0.0, 10.0, 0.5] | 1.0
     
userFeatures este un vector coloană ce conține 3 caracteristici ale unui utilizator. Dorim să combinăm "hour", "mobile" și "userFeatures" în cadrul unui singur vector de caracteristici, denumit "features", pe care să îl folosim pentru a realiza o predicție asupra coloanei "clicked". Pentru aceasta, specificăm coloanele de intrare ale obiectului VectorAssembler ca fiind "hour", "mobile", "userFeatures", iar coloana de ieșire "features". După această transformare, obținem următorul DataFrame:

     id | hour | mobile | userFeatures     | clicked | features
    ----|------|--------|------------------|---------|-----------------------------
     0  | 18   | 1.0    | [0.0, 10.0, 0.5] | 1.0     | [18.0, 1.0, 0.0, 10.0, 0.5]

In [ ]:
#Testati exemplul de mai sus
#Import Vectors, VectorAssembler
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

#Creati DataFrame-ul cu o singura linie de mai sus
dataset = spark.createDataFrame(
    [(0, 18, 1.0, Vectors.dense([0.0, 10.0, 0.5]), 1.0)],
    ["id", "hour", "mobile", "userFeatures", "clicked"])
dataset.show()

+---+----+------+--------------+-------+
| id|hour|mobile|  userFeatures|clicked|
+---+----+------+--------------+-------+
|  0|  18|   1.0|[0.0,10.0,0.5]|    1.0|
+---+----+------+--------------+-------+



In [ ]:
# Creați obiectul VectorAssembler
assembler = VectorAssembler(inputCols=["id", "hour", "mobile", "userFeatures", "clicked"],outputCol="features")

#Transformați datele, aplicând acest asamblor
output = assembler.transform(dataset)

print("Assembled columns 'hour', 'mobile', 'userFeatures' to vector column 'features'")

#Afișați coloanele features și clicked, fără trunchere
output.select('hour', 'mobile', 'userFeatures','features').show(truncate=False)


Assembled columns 'hour', 'mobile', 'userFeatures' to vector column 'features'
+----+------+--------------+-------------------------------+
|hour|mobile|userFeatures  |features                       |
+----+------+--------------+-------------------------------+
|18  |1.0   |[0.0,10.0,0.5]|[0.0,18.0,1.0,0.0,10.0,0.5,1.0]|
+----+------+--------------+-------------------------------+



Celelalte transformări au fost prezentate parțial la curs. Pentru detalii asupra tuturor tipurilor de transformări se va consulta documentația.

In [ ]:
assembler_c=VectorAssembler(inputCols=["groupIndex"],outputCol="features")
output_c = assembler_c.transform(indexed_c)
output_c.show(truncate=False)

+-------+----------+-----+----------+--------+
|Name   |Phone     |Group|groupIndex|features|
+-------+----------+-----+----------+--------+
|John   |4085552424|A    |0.0       |[0.0]   |
|Mike   |3105552738|B    |1.0       |[1.0]   |
|Cassie |4085552424|B    |1.0       |[1.0]   |
|Laura  |3105552438|B    |1.0       |[1.0]   |
|Sarah  |4085551234|A    |0.0       |[0.0]   |
|David  |3105557463|C    |2.0       |[2.0]   |
|Zach   |4085553987|C    |2.0       |[2.0]   |
|Kiera  |3105552938|A    |0.0       |[0.0]   |
|Alexa  |4085559467|C    |2.0       |[2.0]   |
|Karissa|3105553475|A    |0.0       |[0.0]   |
+-------+----------+-----+----------+--------+

